In [1]:
import cymetric as cym
from cymetric import timeseries
import matplotlib as plt 
import pandas as pd
import sys 
import output as oup
sys.path.insert(0, '../../transition-scenarios/scripts/')
import transition_plots as tp
import numpy as np

/home/gwen/.local/lib/python3.6/site-packages/pyne/__init__.py:18: Warning: Error importing PyNE: you should not try to import PyNE from its source directory; please exit the PyNE source tree, and relaunch your python interpreter from there.
  warn(msg, Warning)
/home/gwen/.local/lib/python3.6/site-packages/pyne/__init__.py:18: Warning: Error importing PyNE: you should not try to import PyNE from its source directory; please exit the PyNE source tree, and relaunch your python interpreter from there.
  warn(msg, Warning)
/home/gwen/.local/lib/python3.6/site-packages/pyne/__init__.py:18: Warning: Error importing PyNE: you should not try to import PyNE from its source directory; please exit the PyNE source tree, and relaunch your python interpreter from there.
  warn(msg, Warning)
/home/gwen/.local/lib/python3.6/site-packages/pyne/__init__.py:18: Warning: Error importing PyNE: you should not try to import PyNE from its source directory; please exit the PyNE source tree, and relaunch your 

In [2]:
from cymetric import fco_metrics
from cymetric import filters
from cymetric import timeseries

/home/gwen/.local/lib/python3.6/site-packages/pyne/__init__.py:18: Warning: Error importing PyNE: you should not try to import PyNE from its source directory; please exit the PyNE source tree, and relaunch your python interpreter from there.
  warn(msg, Warning)


In [3]:
starter_string = 'CT'
scenario_nums = ['0','1','2','3','4','5','6','7']

In [4]:
df = oup.initialize_df(scenario_index=starter_string,
                                 scenarios_nums=scenario_nums)

In [5]:
df['Final HLW'] = 0
df['Final Depleted U'] = 0 
df['Total uranium ore'] = 0
df['Total idle capacity'] = 0
df['Last date idle capacity'] = 0
df['Duration of transition'] = 0

In [6]:
df

,Final HLW,Final Depleted U,Total uranium ore,Total idle capacity,Last date idle capacity,Duration of transition
CT,,,,,,
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,0,0,0,0,0,0
7,0,0,0,0,0,0


In [7]:
output_start = '../cyclus-files/oat/cooling-time/ct'
ev_dict = {}
for x in range(len(scenario_nums)): 
    output_file = output_start + scenario_nums[x]+'.sqlite'
    ev_dict[scenario_nums[x]] = cym.Evaluator(db=cym.dbopen(output_file),write=True)

In [ ]:
for x in range(len(scenario_nums)): 
    df.loc[scenario_nums[x],'Final HLW'] = cym.timeseries.transactions(ev_dict[scenario_nums[x]],commodities=['lwrreprocessingwaste','moxreprocessingwaste','frreprocessingwaste'])['Mass'].cumsum().iloc[-1]
    df.loc[scenario_nums[x],'Final Depleted U'] = cym.timeseries.transactions(ev_dict[scenario_nums[x]],commodities=['enrichmentwaste'])['Mass'].cumsum().max()
    df.loc[scenario_nums[x],'Total uranium ore'] = cym.timeseries.transactions(ev_dict[scenario_nums[x]],commodities=['sourceout'])['Mass'].cumsum().max()
    df.loc[scenario_nums[x],'Total idle capacity'], df.loc[scenario_nums[x],'Last date idle capacity'], udict = idlecap(ev_dict[scenario_nums[x]],'(60000+250*t/12)/1000')
    df.loc[scenario_nums[x],'Duration of transition'] = df.loc[scenario_nums[x],'Last date idle capacity'] - 960
    

In [ ]:
df

In [8]:
def idlecap(evaluator,demand_eq): 
    ep = evaluator.eval('FcoMonthlyElectricityGenerated')
    t = np.arange(0,1440)
    power = eval(demand_eq)
    diff = list(ep['Energy'])-power
    total = 0
    undersupply_times = {}
    for x in range(len(diff)):
        if diff[x] < 0: 
            total -= diff[x]
            undersupply_times[x] = -diff[x]
    last_undersupply = max(k for k, v in undersupply_times.items())
    return total, last_undersupply, undersupply_times

In [ ]:
def sensitivity(base_case,init_df):
    """ This function takes a dataframe 
    """
    SA_df = init_df.copy()
    M = init_df.index.size
    categories=list(init_df)
    N = len(categories)
    row = 0
    for x in range(M): 
        if init_df.index[x] == base_case: 
            basecase_index = row
        row += 1
    for x in range(M): 
        if init_df.index[x] == base_case: 
            for y in range(N): 
                SA_df.iloc[x,y] = 0
        else: 
            for y in range(N): 
                if float(init_df.iloc[basecase_index,y]) == 0: 
                    SA_df.iloc[x,y] = np.nan
                else:
                    SA_df.iloc[x,y] = (init_df.iloc[x,y]-init_df.iloc[basecase_index,y])/init_df.iloc[basecase_index,y]*100  
    return SA_df

In [ ]:
sensitivity(base_case='1',init_df=df)

In [ ]:
starter_string = 'CT'
scenario_nums = ['0','1','2','3','4','5','6','7']

In [ ]:
df_p = oup.initialize_df(scenario_index=starter_string,
                                 scenarios_nums=scenario_nums)

In [ ]:
df_p['Max Pu in LWR CP'] = 0
df_p['Max Pu in MOX CP'] = 0 
df_p['Max Pu in SFR CP'] = 0
df_p['Max Pu in all CP'] = 0
df_p['Max Pu in HLW'] = 0
df_p['Max Pu in all RPR'] = 0

In [ ]:
df_p

In [ ]:
output_start = '../cyclus-files/oat/cooling-time/ct'
ev_dict = {}
for x in range(len(scenario_nums)): 
    output_file = output_start + scenario_nums[x]+'-exp.sqlite'
    ev_dict[scenario_nums[x]] = cym.Evaluator(db=cym.dbopen(output_file),write=True)

In [ ]:
for x in range(len(scenario_nums)): 
    df_p.loc[scenario_nums[x],'Max Pu in LWR CP'] = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrstorage'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity'].max()
    df_p.loc[scenario_nums[x],'Max Pu in MOX CP'] = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['moxstorage'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity'].max()
    df_p.loc[scenario_nums[x],'Max Pu in SFR CP'] = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['frstorage'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity'].max()
    df_p.loc[scenario_nums[x],'Max Pu in all CP'] = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrstorage','moxstorage','frstorage'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity'].max()
    df_p.loc[scenario_nums[x],'Max Pu in HLW'] = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['enrichmentsink','lwrsink','moxsink','frsink'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity'].max()
    df_p.loc[scenario_nums[x],'Max Pu in all RPR'] = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrreprocessing','moxreprocessing','frreprocessing'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity'].max()  

In [ ]:
df_p

In [ ]:
sensitivity(base_case='1',init_df=df_p)